In [11]:
import json
from operator import itemgetter
with open("WW_Views.json", encoding="utf-8") as file:
    text = file.read()
    worldwar_wiki_data = json.loads(text)

In [13]:
worldwar_wiki_data

{'Holocaust': [[1050, '2023070100'],
  [1245, '2023070200'],
  [1291, '2023070300'],
  [1242, '2023070400'],
  [1199, '2023070500'],
  [1289, '2023070600'],
  [1114, '2023070700'],
  [1113, '2023070800'],
  [1318, '2023070900'],
  [1501, '2023071000'],
  [1556, '2023071100'],
  [1448, '2023071200'],
  [1443, '2023071300'],
  [1157, '2023071400'],
  [1227, '2023071500'],
  [1270, '2023071600'],
  [1480, '2023071700'],
  [1357, '2023071800'],
  [1415, '2023071900'],
  [1100, '2023072000'],
  [1289, '2023072100'],
  [1094, '2023072200'],
  [1751, '2023072300'],
  [1316, '2023072400'],
  [1511, '2023072500'],
  [1359, '2023072600'],
  [1065, '2023072700'],
  [936, '2023072800'],
  [1141, '2023072900'],
  [915, '2023073000'],
  [1177, '2023073100'],
  [1079, '2023080100'],
  [1308, '2023080200'],
  [1298, '2023080300'],
  [1335, '2023080400'],
  [1313, '2023080500'],
  [1164, '2023080600'],
  [1085, '2023080700'],
  [1078, '2023080800'],
  [1132, '2023080900'],
  [1400, '2023081000'],
  [10

In [5]:
with open("./Data/WW_BacklinkOverview.json", encoding="utf-8") as file:
    text = file.read()
    links = json.loads(text)

In [6]:
links

{'Holocaust': ['Holocaustleugnung',
  'Netti Boleslav',
  'Polizei-Bataillon 320',
  'Léon Ashkénasi',
  'Kindertransport',
  'Jacques Ellul',
  'Adolf Hitlers Politisches Testament',
  'Lucien Steinberg',
  'Ruth Elias',
  'Joseph Hahn (Lyriker)',
  'Ernst Feigl',
  'Kurt Epstein',
  'László Ferenczy',
  'Agneša Kalinová',
  'Simon Wiesenthal',
  'Befehlshaber des Rückwärtigen Heeresgebietes',
  'Polizei-Bataillon 306',
  'Polizei-Bataillon 316',
  'Franz Rademacher (Diplomat)',
  'Arisierung',
  'Wolfgang Seibel (Verwaltungswissenschaftler)',
  'Heinrich Himmler',
  'Walter Labs',
  'Pavel Fantl',
  'Liste der Stolpersteine in Meran',
  'Felix Kolmer',
  'Rita Rosani',
  'Petr Ginz',
  'Franz Murer',
  'Hugo Feigl',
  'Wilhelm Bock (SS-Mitglied)',
  'Schutzstaffel',
  'Jiří Kosta',
  'Geschichte der Juden in Ungarn',
  'Vernichtungslager Sobibor',
  'Politisches Testament von Adolf Hitler',
  'Lucie Dreyfus',
  'Josef Korbel',
  'Lechi',
  'André Trocmé',
  'Anny Engelmann',
  'Mémor

In [14]:
[text_j[value.replace(" ", "_")] for value in links["Holocaust"]]

[[[286, '2023070100'],
  [319, '2023070200'],
  [319, '2023070300'],
  [318, '2023070400'],
  [266, '2023070500'],
  [283, '2023070600'],
  [241, '2023070700'],
  [250, '2023070800'],
  [274, '2023070900'],
  [271, '2023071000'],
  [283, '2023071100'],
  [271, '2023071200'],
  [281, '2023071300'],
  [315, '2023071400'],
  [221, '2023071500'],
  [243, '2023071600'],
  [231, '2023071700'],
  [244, '2023071800'],
  [217, '2023071900'],
  [241, '2023072000'],
  [247, '2023072100'],
  [211, '2023072200'],
  [214, '2023072300'],
  [251, '2023072400'],
  [268, '2023072500'],
  [284, '2023072600'],
  [239, '2023072700'],
  [236, '2023072800'],
  [295, '2023072900'],
  [341, '2023073000'],
  [266, '2023073100'],
  [287, '2023080100'],
  [268, '2023080200'],
  [250, '2023080300'],
  [230, '2023080400'],
  [215, '2023080500'],
  [386, '2023080600'],
  [287, '2023080700'],
  [332, '2023080800'],
  [272, '2023080900'],
  [267, '2023081000'],
  [261, '2023081100'],
  [256, '2023081200'],
  [285, '20

In [11]:
from SampleCreation import getOrderedLinks
import re



NameError: name 're' is not defined

In [93]:
import json
import re
import requests
import urllib3
from bs4 import BeautifulSoup

#LINKS

def getIncomingLinks(wikipage):
    backlinks = []
    session = requests.Session()
    url = "https://de.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": wikipage,
        "bllimit": 500
    }
    lastContinue = {}
    while True:
        actual_param = params.copy()
        actual_param.update(lastContinue)
        response = session.get(url=url, params=actual_param)
        data = response.json()
        if 'error' in data:
            raise Exception(data['error'])
        if 'warnings' in data:
            print(data['warnings'])
        if 'query' in data:
            backlinks.extend(data["query"]["backlinks"])
        if 'continue' not in data:
            break
        lastContinue = data['continue']
    return [backlink["title"] for backlink in backlinks if backlink["title"] is not None]

def getOrderedLinks(wikipage):
    http = urllib3.PoolManager()
    response = http.request("GET", wikipage)
    soup = BeautifulSoup(response.data)
    soup = soup.find(id="mw-content-text")
    link_list = soup.find_all("a",href=re.compile("/wiki/"))
    return link_list

def filterTags(tagList):
    return [e for e in tagList if checkValue(e.get("href")) and e.get("title") is not None]

def filterLinks(linkList):
    return [e for e in linkList if checkValue(e)]

def checkValue(link):
    return (not "Wikipedia:" in link 
        and not "Hilfe:" in link 
        and not "Benutzer:" in link 
        and not "Spezial:" in link 
        and not "Diskussion:" in link 
        and not "Portal:" in link
        and not "Vorlage:" in link
        and not "Kategorie:" in link
        and link is not None)


def tagToTitle(tagList):
    return [i.get("title") for i in tagList]

def deleteDuplicates(titleList):
    return list(dict.fromkeys(titleList))

def getTitlePosition(titleList, title):
    try:
        return titleList.index(title)
    except:
        return 200000

def saveLinks(base_link, outgoing_links, position):
    data = {
        "baselink": base_link,
        "outgoing_links": outgoing_links
    }
    with open(position, "a") as link_file:
        file = json.dumps(data)
        type(file)
        link_file.write(file)

def getPositionsOfBaselink(backlink, baselink):
    orderedBacklinkLinks = deleteDuplicates(tagToTitle(filterTags(getOrderedLinks("https://de.wikipedia.org/wiki/" + backlink))))
    out =  getTitlePosition(orderedBacklinkLinks, baselink)
    if out == 200000:
        print(backlink)
    return out

def createLinkFile(titelList,position):
    for titel in titelList:
        ordered = deleteDuplicates(tagToTitle(filterTags(getOrderedLinks("https://de.wikipedia.org/wiki/" + titel)), 30))
        saveLinks(titel,ordered,position)

#PAGEVIEWS
def getPageViews(name):
    name = name.replace("/","%2f")
    address = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/" + name + "/daily/20230701/20231001"
    # Personal username for identification for the Wikipedia API
    headers = {'User-Agent': 'marek.seipel@web.de'}
    resp = requests.get(address, headers=headers)
    details = resp.json()
    try:
      return [{"date": e["timestamp"], "views": e["views"]} for e in details["items"]]
    except:
      print(name)
      return details


#TOTAL

def createCompleteFile(pageName, location):
    base_views = getPageViews(pageName)
    all_backlinks = filterLinks(getIncomingLinks(pageName))
    backlinks_complete = []
    links_complete = []
    for backlink in all_backlinks:
        backlinks_complete.append({"name": backlink, "position": getPositionsOfBaselink(backlink, pageName), "views": getPageViews(backlink)})
    ordered_links = deleteDuplicates(tagToTitle(filterTags(getOrderedLinks("https://de.wikipedia.org/wiki/" + pageName))))
    for link in ordered_links:
        links_complete.append({"name": link, "views": getPageViews(link)})
    data = {
        "baselink": {
            "name": pageName,
            "views": base_views
        },
        "backlinks": backlinks_complete,
        "links": links_complete
    }
    with open(location, "a") as link_file:
        file = json.dumps(data)
        type(file)
        link_file.write(file)

In [94]:
createCompleteFile("Sankt Augustin","Sankt_Augustin.json")

BFE+
Anthropos (Zeitschrift)
Scheiffart von Merode
Wappen der Stadt Sankt Augustin
Deutschlandkarte, Position der Stadt Sankt Augustin hervorgehoben
Wappen von Sankt Augustin
Bahnhof Bonn-Vilich
Wappen des Rhein-Sieg-Kreises


In [71]:
#getPositionsOfBaselink("Große Anakonda","Ordnung_(Biologie)")
#incoming = getIncomingLinks("Bonn")
len(incoming)

['Liste von Unternehmen mit Namensherkunftserklärungen',
 'Angela Merkel',
 'Anne Haigis',
 'Ampelkoalition',
 'Annette von Droste-Hülshoff',
 'Amnesty International',
 'Baden-Württemberg',
 'Bielefeld',
 'Geschichte von Bündnis 90/Die Grünen',
 'Berliner Mauer',
 'Bremen',
 'Baseball',
 'Bundeskanzler (Deutschland)',
 'Bonn',
 'Bad Homburg vor der Höhe',
 'Bildschirmtext',
 'Bundesregierung (Deutschland)',
 'Dortmund',
 'Dresden',
 'Darmstadt',
 'Düsseldorf',
 'Erich Honecker',
 'Wikipedia:Exzellente Artikel',
 'Franz Müntefering',
 'Freie Demokratische Partei',
 'Greenpeace',
 'Globale Erwärmung',
 'Göttingen',
 'Geschichte Deutschlands',
 'Hamburg',
 'Helmut Kohl',
 'Heroin',
 'Hauptstadt',
 'Hans Bemmann',
 'Heinrich Hertz',
 'Jürgen Habermas',
 'Köln',
 'Konrad Adenauer',
 'Benutzer:Kubi/Beiträge',
 'Leipzig',
 'Ludwig Erhard',
 'Liste von Gewerkschaften in Deutschland',
 'Ludwigshafen am Rhein',
 'Maria Schrader',
 'München',
 'Magdeburg',
 'Mönchengladbach',
 'Nordrhein-Westfale

In [76]:
string = 'Geschichte von Bündnis 90/Die Grünen' #%2f
string.replace("/","%2f")

'Geschichte von Bündnis 90%2fDie Grünen'